# Regressão Linear para Previsão
**Tarefa da Semana:** Implementar um processo de previsão de valores quantitativos utilizando regressão linear, com base em um dataset escolhido pelo grupo ou no dataset fornecido.

**Grupo**
| Nome | RA |
| ---- | -- |
| Luiz Gabriel Profirio Mendes | 10382703 |



## 1. Definição da Variável Alvo

A variável alvo será ``median_house_value``.  
A relevância da predição dessa variável, pode ser relevante para estimar o valor médio de um novo imóvel adicionado ao dataset.   
Em aplicações práticas, essa predição poderia ser utilizada para auxiliar investidores imobiliários que estão a procura de oportunidades de negócio no mercado. Com essa predição, poderiam descobrir se estão pagando um valor baixo, normal ou alto, para um determinado imóvel e usar essas informações para decidir se realizarão uma compra ou não.



In [25]:
# Importa bibliotecas
import pandas as pd

In [26]:
# Lê CSV
df = pd.read_csv("aula4_housingprices.csv")

In [27]:
# Exibe algumas informações básicas sobre o dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


### Detecção de valores null

A variável ``total_bedrooms`` possui 207 valores nulos e, para preservar a integridade da análise, irei tratá-los com a interpolação.  

In [28]:
# Exibe as linhas do dataset que possuem NaN na coluna total_bedrooms.
display(df.query("total_bedrooms.isnull()"))

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
290,-122.16,37.77,47.0,1256.0,NaN,570.0,218.0,4.3750,161900.0,NEAR BAY
341,-122.17,37.75,38.0,992.0,NaN,732.0,259.0,1.6196,85100.0,NEAR BAY
538,-122.28,37.78,29.0,5154.0,NaN,3741.0,1273.0,2.5762,173400.0,NEAR BAY
563,-122.24,37.75,45.0,891.0,NaN,384.0,146.0,4.9489,247100.0,NEAR BAY
696,-122.10,37.69,41.0,746.0,NaN,387.0,161.0,3.9063,178400.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20267,-119.19,34.20,18.0,3620.0,NaN,3171.0,779.0,3.3409,220500.0,NEAR OCEAN
20268,-119.18,34.19,19.0,2393.0,NaN,1938.0,762.0,1.6953,167400.0,NEAR OCEAN
20372,-118.88,34.17,15.0,4260.0,NaN,1701.0,669.0,5.1033,410700.0,<1H OCEAN
20460,-118.75,34.29,17.0,5512.0,NaN,2734.0,814.0,6.6073,258100.0,<1H OCEAN


In [29]:
# Preenche valores faltantes com base em valores próximos
df['total_bedrooms'] = df['total_bedrooms'].interpolate()

# Exibe algumas das colunas que anteriormente estavam com valores NaN.
display(df.loc[[290, 341, 538, 563, 696]])

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
290,-122.16,37.77,47.0,1256.0,161.0,570.0,218.0,4.3750,161900.0,NEAR BAY
341,-122.17,37.75,38.0,992.0,239.5,732.0,259.0,1.6196,85100.0,NEAR BAY
538,-122.28,37.78,29.0,5154.0,338.5,3741.0,1273.0,2.5762,173400.0,NEAR BAY
563,-122.24,37.75,45.0,891.0,521.0,384.0,146.0,4.9489,247100.0,NEAR BAY
696,-122.10,37.69,41.0,746.0,720.0,387.0,161.0,3.9063,178400.0,NEAR BAY


In [30]:
# Exibe algumas informações básicas sobre o dataset.
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


None

### Tratamento da coluna ``ocean_proximity``

Para realizar o treinamento do nosso modelo, devemos ter apenas valores numéricos nessa coluna. Como essa é uma coluna que contém dados que impactam diretamente no preço de um imóvel, não podemos simplesmente removê-la.

In [31]:
# Verifica quais são os valores possíveis para essa coluna
display(df['ocean_proximity'].value_counts())

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64

A seguinte conversão de valores será aplicada para essa coluna:

| ocean_proximity | valor |
| --------------- | ----- |
| <1H OCEAN | 0 |
| INLAND | 1 |
| NEAR OCEAN | 2 |
| NEAR BAY | 3 |
| ISLAND | 4 |

In [32]:
# Substitui as strings pelos valores numéricos.
df.loc[(df['ocean_proximity'] == '<1H OCEAN'), 'ocean_proximity'] = 0
df.loc[(df['ocean_proximity'] == 'INLAND'), 'ocean_proximity'] = 1
df.loc[(df['ocean_proximity'] == 'NEAR OCEAN'), 'ocean_proximity'] = 2
df.loc[(df['ocean_proximity'] == 'NEAR BAY'), 'ocean_proximity'] = 3
df.loc[(df['ocean_proximity'] == 'ISLAND'), 'ocean_proximity'] = 4


In [33]:
# Verifica quais são os valores possíveis para essa coluna
display(df['ocean_proximity'].value_counts())

ocean_proximity
0    9136
1    6551
2    2658
3    2290
4       5
Name: count, dtype: int64

## 2. Divisão do Conjunto de Dados
Separaremos 80% dos dados para treino e 20% para teste.  
Uma observação importante é que, analisando as colunas do dataset e os dados contidos nelas, podemos concluir que todas as colunas possuem dados que são relevantes para determinar o valor de um imóvel. Por isso, nesse caso, não realizaremos a remoção de nenhuma coluna.

In [34]:
# Importa train_test_split
from sklearn.model_selection import train_test_split

# Defino como x(features), todas as colunas do meu dataset, com excessão da minha variável alvo.
x = df.drop(columns=['median_house_value'])

# Defino como y(target) a coluna median_house_value. 
y = df['median_house_value']

# Separamos os dados de teste como 20% e, consequentemente, os de treino como 80%.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(y_train.shape)
print(x_train.shape)

(16512,)
(16512, 9)


## 3. Treinamento do Modelo de Regressão Linear

In [35]:
# Importa LinearRegression.
from sklearn.linear_model import LinearRegression

# Inicializa o modelo de regressão linear.
modelo = LinearRegression()

# Treinamos o modelo com os dados de teste obtidos anteriormente.
modelo.fit(x_train, y_train)

LinearRegression()

## 4. Avaliação do Modelo

Calculamos Erro Quadrático Médio (MSE) e Coeficiente de Determinação (R²).

In [36]:
# Prevê valores com base no conjunto de teste
y_pred = modelo.predict(x_test)

In [37]:
from sklearn.metrics import mean_squared_error

# Calcula o Erro Quadrático Médio (MSE)
mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")

MSE: 5073245512.558103


In [38]:
from sklearn.metrics import r2_score

# Calcula o Coeficiente de Determinação (R²)
r2 = r2_score(y_test, y_pred)

print(f"R²: {r2}")

R²: 0.6128502172376402


O valor R² obtido é bom, pois é mais próximo de 1, indicando que o modelo explica bem os dados. No entanto, o valor de MSE é extremamente alto, indicando que o modelo tem um grande erro médio, ou seja, está errando muito.  

Tendo em vista que principalmente o valor de MSE está muito fora do esperado, poderíamos tentar melhorar os resultados ao buscar e remover outliers e normalizar os dados presentes no dataset fazendo ajustes de escalas, por exemplo.